In [13]:
!pip install tf-nightly==2.4.0-dev20200929

In [14]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [15]:
import tensorflow as tf
tf.__version__

'2.4.0-dev20200929'

In [16]:
%cd /gdrive/'My Drive'/EfficientNetD0/ 
%ls 'fine_tuned_model/saved_model'
saved_model_dir = 'fine_tuned_model/saved_model'


model = tf.saved_model.load(saved_model_dir)
model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY].inputs[0].set_shape([1, 512, 512, 3])
tf.saved_model.save(model, "saved_model_new", signatures=model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY])

/gdrive/My Drive/EfficientNetD0
assets/  saved_model.pb  variables/
INFO:tensorflow:Assets written to: saved_model_new/assets


In [17]:
# Convert
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='saved_model_new', signature_keys=['serving_default'])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

In [18]:
%pwd

'/gdrive/My Drive/EfficientNetD0'

In [19]:
import numpy as np


## TFLite Interpreter to check input shape
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print input and output tensors
print("INPUT DETAILS: \n",input_details)
print("OUTPUT DETAILS: \n",output_details)
# Test the model on random input data.
input_shape = input_details[0]['shape']
print(input_shape)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

INPUT DETAILS: 
 [{'name': 'serving_default_input_tensor:0', 'index': 0, 'shape': array([  1, 512, 512,   3], dtype=int32), 'shape_signature': array([  1, 512, 512,   3], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
OUTPUT DETAILS: 
 [{'name': 'StatefulPartitionedCall:4', 'index': 66544, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([ 1, -1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'StatefulPartitionedCall:6', 'index': 66383, 'shape': array([    1, 49104,     4], dtype=int32), 'shape_signature': array([    1, 49104,     4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0

In [20]:

tflite_file = '/gdrive/My Drive/EfficientNetD0/fine_tuned_model/mouthNoseDetector.tflite'
#%cd /drive/My\ Drive/Maithri_TF/fine_tuned_model/
%cd /gdrive/'My Drive'/EfficientNetD0/fine_tuned_model/
#now save the tflite model to the file
#tflite_model.save(tflite_file)   #Note this does not seem to work although in google documentation
open(tflite_file, "wb").write(tflite_model)
!ls /gdrive/'My Drive'/EfficientNetD0/fine_tuned_model

/gdrive/My Drive/EfficientNetD0/fine_tuned_model
checkpoint  mouthNoseDetector.tflite  pipeline.config  saved_model


In [21]:
from google.colab import files
#files.download(tflite_model)

!ls /gdrive/'My Drive'/EfficientNetD0/fine_tuned_model

files.download('/gdrive/My Drive/EfficientNetD0/fine_tuned_model/mouthNoseDetector.tflite')

checkpoint  mouthNoseDetector.tflite  pipeline.config  saved_model


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
import random
import cv2
import glob
import numpy as np
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path='/gdrive/My Drive/EfficientNetD0/fine_tuned_model/mouthNoseDetector.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


### LOADING IMAGE
'''
def load_grayscale_image_into_numpy_array(path):
  # read in image as grayscale cv2 image
  img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  # convert image to rgb
  backtorgb = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
  return backtorgb.astype(np.uint8)
'''
def load_rgb_image_into_numpy_array(path):
  # read in image as grayscale cv2 image
  img = cv2.imread(path, cv2.IMREAD_COLOR)
  
  im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return im_rgb

# download test image
test_image = load_rgb_image_into_numpy_array('/gdrive/My Drive/EfficientNetD0/inference/testImages/4.jpg')

# RESIZE TO 512 x 512

# set a new height in pixels
new_height = 512
new_width = 512
dsize = (new_width, new_height)
# resize image
scaled_image = cv2.resize(test_image, dsize, interpolation = cv2.INTER_AREA)
# generate tensor
input_tensor = tf.convert_to_tensor(np.expand_dims(scaled_image, 0), dtype=tf.uint8)

print(input_tensor.shape)

interpreter.set_tensor(input_details[0]['index'], input_tensor)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)
print("output array length: " + str(len(output_data[0])))

(1, 512, 512, 3)
[[0.9152355  0.04657206 0.03623417 0.02410164 0.0208689  0.0208689
  0.02061626 0.02006075 0.01974621 0.01739538 0.01652172 0.01646337
  0.01644087 0.01619524 0.0158782  0.01430142 0.01422951 0.01392865
  0.01340008 0.01322415 0.01206633 0.01189646 0.01174569 0.01128125
  0.01076326 0.01061738 0.01051983 0.01047349 0.01033157 0.01032251
  0.01024464 0.00984514 0.00982162 0.009736   0.00933856 0.00923952
  0.00909239 0.00909001 0.00895402 0.00887522 0.00883248 0.00865069
  0.00858086 0.00831637 0.00826919 0.0081026  0.00784248 0.00761989
  0.00759077 0.00757471 0.00751126 0.00740901 0.00730228 0.00695211
  0.00688955 0.006868   0.00678909 0.00666997 0.00664672 0.00650692
  0.00642684 0.00617179 0.00616792 0.00610325 0.00607902 0.00577149
  0.00573507 0.00562155 0.00552177 0.00540039 0.00535631 0.00532612
  0.00527304 0.00524318 0.00522834 0.0051991  0.00512895 0.00511232
  0.00509009 0.00500232 0.00487533 0.00487322 0.00483268 0.00480166
  0.00479409 0.0047549  0.004674